<a href="https://colab.research.google.com/github/bipinKrishnan/torchkeras/blob/master/Model_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torch import nn

In [89]:
class Input:
  def __init__(self, shape):
    self.size = shape

In [162]:
class Dense(Input):
  def __init__(self, outputs, activation):
    self.outputs = outputs
    self.activation = activation

  def __call__(self, input_size):
    self.inputs_size = input_size.size
    self.layers = nn.Sequential(
        nn.Linear(self.inputs_size, self.outputs),
        self.activation
    )

    return self.layers

class Dense_():
  def __init__(self, outputs, activation):
    self.outputs = outputs
    self.activation = activation

  def __call__(self, inputs):
    self.inputs = inputs
    self.layers = list(inputs)
    self.layers.extend([nn.Linear(self.layers[0].out_features, self.outputs), self.activation])

    self.layers = nn.Sequential(*self.layers)

    return self.layers


In [163]:
input = Input(shape=9)
x = Dense(46, nn.ReLU())(input)
y = Dense_(34, nn.ReLU())(x)
y

Sequential(
  (0): Linear(in_features=9, out_features=46, bias=True)
  (1): ReLU()
  (2): Linear(in_features=46, out_features=34, bias=True)
  (3): ReLU()
)